<a href="https://colab.research.google.com/github/PraewLao/price-and-peers-app/blob/main/Revise_train_and_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

# Load the datasets
file_35 = "/content/GICS_35_stockreturns_2000-2024.xlsx"
file_25 = "/content/GICS_25_stockreturns_2000-2024.xlsx"
file_45 = "/content/GICS_45_stockreturns_2000-2024.xlsx"
file_factors = "/content/FF 3 factors and MOM 2000-2024.xlsx"

# Read stock return data
returns_35 = pd.read_excel(file_35)
returns_25 = pd.read_excel(file_25)
returns_45 = pd.read_excel(file_45)

# Read factor data
factors_df = pd.read_excel(file_factors)

# Display sample data for verification
returns_35.head(), returns_25.head(), returns_45.head(), factors_df.head()


(   PERMNO Names Date Ticker Symbol   Returns
 0   10180 2000-01-31          AKRN  0.076923
 1   10180 2000-02-29          AKRN  0.809524
 2   10180 2000-03-31          AKRN  0.000000
 3   10180 2000-04-28          AKRN -0.269737
 4   10180 2000-05-31          AKRN  0.153153,
    PERMNO Names Date Ticker Symbol   Returns
 0   12079 2000-01-31            GM  0.108341
 1   12079 2000-02-29            GM -0.049651
 2   12079 2000-03-31            GM  0.088743
 3   12079 2000-04-28            GM  0.130566
 4   12079 2000-05-31            GM -0.240320,
    PERMNO Names Date Ticker Symbol   Returns
 0   10104 2000-01-31          ORCL -0.108477
 1   10104 2000-02-29          ORCL  0.486393
 2   10104 2000-03-31          ORCL  0.051347
 3   10104 2000-04-28          ORCL  0.024019
 4   10104 2000-05-31          ORCL -0.100860,
    Excess Return on the Market  Small-Minus-Big Return  High-Minus-Low Return  \
 0                      -0.0474                  0.0577                -0.0188   
 1   

In [2]:
# Rename columns and drop missing returns
returns_35_clean = returns_35.dropna(subset=['Returns']).copy()
returns_25_clean = returns_25.dropna(subset=['Returns']).copy()
returns_45_clean = returns_45.dropna(subset=['Returns']).copy()

# Rename columns for consistency
returns_35_clean.rename(columns={'Names Date': 'Date', 'Ticker Symbol': 'Ticker'}, inplace=True)
returns_25_clean.rename(columns={'Names Date': 'Date', 'Ticker Symbol': 'Ticker'}, inplace=True)
returns_45_clean.rename(columns={'Names Date': 'Date', 'Ticker Symbol': 'Ticker'}, inplace=True)

# Convert 'Date' to datetime format
returns_35_clean['Date'] = pd.to_datetime(returns_35_clean['Date'])
returns_25_clean['Date'] = pd.to_datetime(returns_25_clean['Date'])
returns_45_clean['Date'] = pd.to_datetime(returns_45_clean['Date'])

# Clean and rename factor dataset
factors_df.rename(columns={
    'Excess Return on the Market': 'Mkt-RF',
    'Small-Minus-Big Return': 'SMB',
    'High-Minus-Low Return': 'HML',
    'Risk-Free Return Rate (One Month Treasury Bill Rate)': 'RF',
    'Momentum Factor': 'MOM',
    'Date (SAS). Last Trading Day of the Month': 'Date'
}, inplace=True)

factors_df['Date'] = pd.to_datetime(factors_df['Date'])
factors_df.dropna(inplace=True)


In [5]:
# Merge and compute excess return for Healthcare (GICS 35)
healthcare_merged = pd.merge(returns_35_clean, factors_df, on='Date', how='inner')
healthcare_merged['Excess Return'] = healthcare_merged['Returns'] - healthcare_merged['RF']

# Merge and compute excess return for Consumer Discretionary (GICS 25)
consumer_merged = pd.merge(returns_25_clean, factors_df, on='Date', how='inner')
consumer_merged['Excess Return'] = consumer_merged['Returns'] - consumer_merged['RF']

# Merge and compute excess return for Information Technology (GICS 45)
tech_merged = pd.merge(returns_45_clean, factors_df, on='Date', how='inner')
tech_merged['Excess Return'] = tech_merged['Returns'] - tech_merged['RF']

In [10]:
import pandas as pd
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

# === Function to evaluate models with train/test split ===
def evaluate_model_sector_split(df, model_type, split_date='2019-01-01'):
    X_cols = {
        'CAPM': ['Mkt-RF'],
        'FF3': ['Mkt-RF', 'SMB', 'HML'],
        'Carhart': ['Mkt-RF', 'SMB', 'HML', 'MOM']
    }

    results = []

    for ticker in df['Ticker'].unique():
        stock_data = df[df['Ticker'] == ticker].dropna().sort_values('Date')

        if len(stock_data) < 36:
            continue

        train = stock_data[stock_data['Date'] < split_date]
        test = stock_data[stock_data['Date'] >= split_date]

        if len(train) < 24 or len(test) < 12:
            continue

        y_train = train['Excess Return']
        X_train = sm.add_constant(train[X_cols[model_type]])

        y_test = test['Excess Return']
        X_test = sm.add_constant(test[X_cols[model_type]])

        model = sm.OLS(y_train, X_train).fit()
        y_pred_test = model.predict(X_test)

        rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
        mae = mean_absolute_error(y_test, y_pred_test)

        results.append({
            'Ticker': ticker,
            'Model': model_type,
            'Train R2': model.rsquared if model_type == 'CAPM' else None,
            'Train Adj_R2': model.rsquared_adj if model_type != 'CAPM' else None,
            'Test RMSE': rmse,
            'Test MAE': mae,
            'Train Obs': len(train),
            'Test Obs': len(test)
        })

    return pd.DataFrame(results)

# === Run models for Healthcare sector ===
capm_healthcare = evaluate_model_sector_split(healthcare_merged, 'CAPM')
ff3_healthcare = evaluate_model_sector_split(healthcare_merged, 'FF3')
carhart_healthcare = evaluate_model_sector_split(healthcare_merged, 'Carhart')

# Combine all results
healthcare_out_of_sample = pd.concat(
    [capm_healthcare, ff3_healthcare, carhart_healthcare],
    ignore_index=True
)

# === Summarize model performance ===
summary_stats = healthcare_out_of_sample.groupby("Model").agg({
    "Train R2": "mean",
    "Train Adj_R2": "mean",
    "Test RMSE": "mean",
    "Test MAE": "mean",
    "Train Obs": "sum",
    "Test Obs": "sum"
}).reset_index()

# Determine best models
best_model_rmse = summary_stats.loc[summary_stats["Test RMSE"].idxmin(), "Model"]
best_model_mae = summary_stats.loc[summary_stats["Test MAE"].idxmin(), "Model"]

# === Display results ===
print("=== Average Model Performance: Healthcare Sector ===")
print(summary_stats.round(4))
print("\n✅ Best model by Test RMSE:", best_model_rmse)
print("✅ Best model by Test MAE:", best_model_mae)

=== Average Model Performance: Healthcare Sector ===
     Model  Train R2  Train Adj_R2  Test RMSE  Test MAE  Train Obs  Test Obs
0     CAPM    0.1183           NaN     0.1853    0.1268      21415     10132
1  Carhart       NaN        0.1599     0.1905    0.1310      21415     10132
2      FF3       NaN        0.1549     0.1886    0.1295      21415     10132

✅ Best model by Test RMSE: CAPM
✅ Best model by Test MAE: CAPM


<ipython-input-10-57ee75a82704>:59: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  healthcare_out_of_sample = pd.concat(


In [11]:
# === Function reused from before ===
def evaluate_model_sector_split(df, model_type, split_date='2019-01-01'):
    X_cols = {
        'CAPM': ['Mkt-RF'],
        'FF3': ['Mkt-RF', 'SMB', 'HML'],
        'Carhart': ['Mkt-RF', 'SMB', 'HML', 'MOM']
    }

    results = []

    for ticker in df['Ticker'].unique():
        stock_data = df[df['Ticker'] == ticker].dropna().sort_values('Date')

        if len(stock_data) < 36:
            continue

        train = stock_data[stock_data['Date'] < split_date]
        test = stock_data[stock_data['Date'] >= split_date]

        if len(train) < 24 or len(test) < 12:
            continue

        y_train = train['Excess Return']
        X_train = sm.add_constant(train[X_cols[model_type]])

        y_test = test['Excess Return']
        X_test = sm.add_constant(test[X_cols[model_type]])

        model = sm.OLS(y_train, X_train).fit()
        y_pred_test = model.predict(X_test)

        rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
        mae = mean_absolute_error(y_test, y_pred_test)

        results.append({
            'Ticker': ticker,
            'Model': model_type,
            'Train R2': model.rsquared if model_type == 'CAPM' else None,
            'Train Adj_R2': model.rsquared_adj if model_type != 'CAPM' else None,
            'Test RMSE': rmse,
            'Test MAE': mae,
            'Train Obs': len(train),
            'Test Obs': len(test)
        })

    return pd.DataFrame(results)

# === Run models for Consumer Discretionary sector ===
capm_consumer = evaluate_model_sector_split(consumer_merged, 'CAPM')
ff3_consumer = evaluate_model_sector_split(consumer_merged, 'FF3')
carhart_consumer = evaluate_model_sector_split(consumer_merged, 'Carhart')

# Combine all results
consumer_out_of_sample = pd.concat(
    [capm_consumer, ff3_consumer, carhart_consumer],
    ignore_index=True
)

# === Summarize model performance ===
summary_stats_consumer = consumer_out_of_sample.groupby("Model").agg({
    "Train R2": "mean",
    "Train Adj_R2": "mean",
    "Test RMSE": "mean",
    "Test MAE": "mean",
    "Train Obs": "sum",
    "Test Obs": "sum"
}).reset_index()

# Determine best models
best_model_rmse = summary_stats_consumer.loc[summary_stats_consumer["Test RMSE"].idxmin(), "Model"]
best_model_mae = summary_stats_consumer.loc[summary_stats_consumer["Test MAE"].idxmin(), "Model"]

# === Display results ===
print("=== Average Model Performance: Consumer Discretionary Sector ===")
print(summary_stats_consumer.round(4))
print("\n✅ Best model by Test RMSE:", best_model_rmse)
print("✅ Best model by Test MAE:", best_model_mae)


=== Average Model Performance: Consumer Discretionary Sector ===
     Model  Train R2  Train Adj_R2  Test RMSE  Test MAE  Train Obs  Test Obs
0     CAPM    0.1971           NaN     0.1632    0.0989      12687      5075
1  Carhart       NaN        0.2410     0.1620    0.0978      12687      5075
2      FF3       NaN        0.2289     0.1619    0.0979      12687      5075

✅ Best model by Test RMSE: FF3
✅ Best model by Test MAE: Carhart


<ipython-input-11-0e7c182bd311>:54: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  consumer_out_of_sample = pd.concat(


In [12]:
# === Function reused from before ===
def evaluate_model_sector_split(df, model_type, split_date='2019-01-01'):
    X_cols = {
        'CAPM': ['Mkt-RF'],
        'FF3': ['Mkt-RF', 'SMB', 'HML'],
        'Carhart': ['Mkt-RF', 'SMB', 'HML', 'MOM']
    }

    results = []

    for ticker in df['Ticker'].unique():
        stock_data = df[df['Ticker'] == ticker].dropna().sort_values('Date')

        if len(stock_data) < 36:
            continue

        train = stock_data[stock_data['Date'] < split_date]
        test = stock_data[stock_data['Date'] >= split_date]

        if len(train) < 24 or len(test) < 12:
            continue

        y_train = train['Excess Return']
        X_train = sm.add_constant(train[X_cols[model_type]])

        y_test = test['Excess Return']
        X_test = sm.add_constant(test[X_cols[model_type]])

        model = sm.OLS(y_train, X_train).fit()
        y_pred_test = model.predict(X_test)

        rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
        mae = mean_absolute_error(y_test, y_pred_test)

        results.append({
            'Ticker': ticker,
            'Model': model_type,
            'Train R2': model.rsquared if model_type == 'CAPM' else None,
            'Train Adj_R2': model.rsquared_adj if model_type != 'CAPM' else None,
            'Test RMSE': rmse,
            'Test MAE': mae,
            'Train Obs': len(train),
            'Test Obs': len(test)
        })

    return pd.DataFrame(results)

# === Run models for Information Technology sector ===
capm_tech = evaluate_model_sector_split(tech_merged, 'CAPM')
ff3_tech = evaluate_model_sector_split(tech_merged, 'FF3')
carhart_tech = evaluate_model_sector_split(tech_merged, 'Carhart')

# Combine all results
tech_out_of_sample = pd.concat(
    [capm_tech, ff3_tech, carhart_tech],
    ignore_index=True
)

# === Summarize model performance ===
summary_stats_tech = tech_out_of_sample.groupby("Model").agg({
    "Train R2": "mean",
    "Train Adj_R2": "mean",
    "Test RMSE": "mean",
    "Test MAE": "mean",
    "Train Obs": "sum",
    "Test Obs": "sum"
}).reset_index()

# Determine best models
best_model_rmse = summary_stats_tech.loc[summary_stats_tech["Test RMSE"].idxmin(), "Model"]
best_model_mae = summary_stats_tech.loc[summary_stats_tech["Test MAE"].idxmin(), "Model"]

# === Display results ===
print("=== Average Model Performance: Information Technology Sector ===")
print(summary_stats_tech.round(4))
print("\n✅ Best model by Test RMSE:", best_model_rmse)
print("✅ Best model by Test MAE:", best_model_mae)


=== Average Model Performance: Information Technology Sector ===
     Model  Train R2  Train Adj_R2  Test RMSE  Test MAE  Train Obs  Test Obs
0     CAPM     0.258           NaN     0.1044    0.0783       5407      2154
1  Carhart       NaN        0.3050     0.1052    0.0792       5407      2154
2      FF3       NaN        0.2952     0.1051    0.0793       5407      2154

✅ Best model by Test RMSE: CAPM
✅ Best model by Test MAE: CAPM


<ipython-input-12-6b7f6eee654a>:54: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  tech_out_of_sample = pd.concat(
